# 如何从目录加载文档

LangChain 的 [DirectoryLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.DirectoryLoader.html) 实现了将文件从磁盘读取到 LangChain [Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html#langchain_core.documents.base.Document) 对象的功能。在此，我们将演示：

- 如何从文件系统加载，包括使用通配符模式；
- 如何使用多线程进行文件 I/O；
- 如何使用自定义加载器类来解析特定文件类型（例如，代码）；
- 如何处理错误，例如由于解码引起的错误。

In [1]:
from langchain_community.document_loaders import DirectoryLoader

`DirectoryLoader` 接受一个名为 `loader_cls` 的关键字参数，它默认为 [UnstructuredLoader](/docs/integrations/document_loaders/unstructured_file)。[Unstructured](https://unstructured-io.github.io/unstructured/) 支持解析多种格式，例如 PDF 和 HTML。在这里，我们使用它来读取一个 markdown（.md）文件。

我们可以使用 `glob` 参数来控制要加载哪些文件。请注意，这里它不会加载 `.rst` 文件或 `.html` 文件。

In [2]:
loader = DirectoryLoader("../", glob="**/*.md")
docs = loader.load()
len(docs)

20

In [3]:
print(docs[0].page_content[:100])

Security

LangChain has a large ecosystem of integrations with various external resources like local


## 显示进度条

默认情况下，不显示进度条。要显示进度条，请安装 `tqdm` 库（例如 `pip install tqdm`），并将 `show_progress` 参数设置为 `True`。

In [4]:
loader = DirectoryLoader("../", glob="**/*.md", show_progress=True)
docs = loader.load()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 54.56it/s]


## 使用多线程

默认情况下，加载发生在单个线程中。为了利用多个线程，将 `use_multithreading` 标志设置为 true。

In [5]:
loader = DirectoryLoader("../", glob="**/*.md", use_multithreading=True)
docs = loader.load()

## 更改加载器类
默认情况下，此功能使用 `UnstructuredLoader` 类。要自定义加载器，请在 `loader_cls` 关键字参数中指定加载器类。下面我们展示一个使用 [TextLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.text.TextLoader.html) 的示例：

In [6]:
from langchain_community.document_loaders import TextLoader

loader = DirectoryLoader("../", glob="**/*.md", loader_cls=TextLoader)
docs = loader.load()

In [7]:
print(docs[0].page_content[:100])

# Security

LangChain has a large ecosystem of integrations with various external resources like loc


请注意，虽然 `UnstructuredLoader` 可以解析 Markdown 标题，但 `TextLoader` 不能。

如果你需要加载 Python 源代码文件，请使用 `PythonLoader`：

In [8]:
from langchain_community.document_loaders import PythonLoader

loader = DirectoryLoader("../../../../../", glob="**/*.py", loader_cls=PythonLoader)

## 使用 TextLoader 自动检测文件编码

`DirectoryLoader` 可以帮助管理因文件编码差异而导致的错误。下面我们将尝试加载一组文件，其中一个文件包含非 UTF-8 编码。

In [9]:
path = "../../../libs/langchain/tests/unit_tests/examples/"

loader = DirectoryLoader(path, glob="**/*.txt", loader_cls=TextLoader)

### A. 默认行为

默认情况下，我们会抛出错误：

In [10]:
loader.load()

Error loading file ../../../../libs/langchain/tests/unit_tests/examples/example-non-utf8.txt


RuntimeError: Error loading ../../../../libs/langchain/tests/unit_tests/examples/example-non-utf8.txt

文件 `example-non-utf8.txt` 使用了不同的编码，因此 `load()` 函数会失败并显示一条有用的消息，指出哪个文件解码失败。

使用 `TextLoader` 的默认行为，任何无法加载的文档都会导致整个加载过程失败，并且不会加载任何文档。

### B. 静默失败

我们可以将 `silent_errors` 参数传递给 `DirectoryLoader`，以跳过无法加载的文件并继续加载过程。

In [11]:
loader = DirectoryLoader(
    path, glob="**/*.txt", loader_cls=TextLoader, silent_errors=True
)
docs = loader.load()

Error loading file ../../../../libs/langchain/tests/unit_tests/examples/example-non-utf8.txt: Error loading ../../../../libs/langchain/tests/unit_tests/examples/example-non-utf8.txt


In [12]:
doc_sources = [doc.metadata["source"] for doc in docs]
doc_sources

['../../../../libs/langchain/tests/unit_tests/examples/example-utf8.txt']

### C. 自动检测编码

我们也可以通过将 `autodetect_encoding` 传递给加载器类，来要求 `TextLoader` 在失败前自动检测文件编码。

In [13]:
text_loader_kwargs = {"autodetect_encoding": True}
loader = DirectoryLoader(
    path, glob="**/*.txt", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs
)
docs = loader.load()

In [14]:
doc_sources = [doc.metadata["source"] for doc in docs]
doc_sources

['../../../../libs/langchain/tests/unit_tests/examples/example-utf8.txt',
 '../../../../libs/langchain/tests/unit_tests/examples/example-non-utf8.txt']